<h1>Imports of the necesary libraries</h1>

In [56]:
!pip install geocoder
!pip install geopy
!pip install folium

In [57]:
import geocoder
import pandas as pd
import numpy as np
import requests
import urllib.request
import matplotlib.pyplot as plt
import sklearn.utils
import folium
from sklearn.cluster import KMeans
from geopy.geocoders import Nominatim
from bs4 import BeautifulSoup
from sklearn.cluster import DBSCAN 
from sklearn.datasets.samples_generator import make_blobs 
from sklearn.preprocessing import StandardScaler 
from pylab import rcParams
import matplotlib.cm as cm
import matplotlib.colors as colors
%matplotlib inline

<h1>Load information from Wikipedia web page</h1>

In [58]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
response = requests.get(url)

<h1>Parse webpage to get only the information table</h1>

to get the information table the library bs4 was used, this library allows read the HTML of the webpage, query the content, and extract information using HTML tags.

In [59]:
soup = BeautifulSoup(response.text, 'html.parser')
table_data = soup.find_all('table')[0]
table_data
df = pd.read_html(table_data.prettify(), flavor='bs4')[0]
df.head()

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


<h1>Drop all Borough rows that has value 'Not assigned' </h1>

In [60]:
df = df[df.Borough != 'Not assigned']
df.head()

,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


<h1>Remove Neighborhood with 'Not assigned' value </h1>

Check if Neighborhood with value 'Not assigned' exist and copy the value of the Borough column on the rows that has it.

In [61]:
df_aux = df[df.Borough == 'Neighborhood']
if(df_aux.size>0):
    df.loc[lambda df: df['Neighborhood'] == 'Not assigned', df['Borough']]

df.head()

,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [62]:
df.shape

(103, 3)

<h1>Load data from csv file</h1>

In [63]:
#load csv with geographical data
import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_c51e6280f7c446268db4cb1c6b3c607c = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='6nnWxMA7gg8_NXb6OVR_9hQKnRWXhk-SoI9Ax--6tIEv',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.eu-geo.objectstorage.service.networklayer.com')

body = client_c51e6280f7c446268db4cb1c6b3c607c.get_object(Bucket='applieddatasciencecapstone-donotdelete-pr-vkj5esor2efmht',Key='Geospatial_Coordinates.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

geo_data = pd.read_csv(body)
geo_data.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


<h1>Merge geographical info with wikipedia info </h1>

In [64]:
# Merge lat and long with the data from wikipedia
geo_data
df = pd.merge(df, geo_data, on='Postal Code')
df.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [65]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(df['Borough'].unique()),
        df.shape[0]
    )
)

The dataframe has 10 boroughs and 103 neighborhoods.


<h1>Alternatively, geographical info can be obtained with the geolocator</h1>

In [66]:
# Center map in toronto
address = 'Toronto'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 43.6534817, -79.3839347.


<h1>Create a map with the result of merge location data and wikipedia neighborhoods data</h1>

In [67]:
#Create map using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [68]:
def getNearbyVenues(names, latitudes, longitudes, radius=500, LIMIT=100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        #print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

<h1>Use foursquare to get venues information for each neighborhood</h1>

In [69]:
CLIENT_ID = 'FFRJN0UR5SSHZTTQIBYXKNEDJXKKPPBVTMK3OJIBVD02JDG3' # your Foursquare ID
CLIENT_SECRET = 'I2SZ4OU2ZFUWPRPXOIAVXUROGNJDXCRE5WTPZDVJRL42E1E1' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

toronto_venues = getNearbyVenues(names=df['Neighborhood'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )
print(toronto_venues.shape)
toronto_venues.head()

(2136, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
2,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
3,Victoria Village,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop
4,Victoria Village,43.725882,-79.315572,Portugril,43.725819,-79.312785,Portuguese Restaurant


In [70]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 271 uniques categories.


<h1>Create categorical information for each neighborhood and his respective venues </h1>

In [71]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# get Neighborhood column index
NhIndex = toronto_onehot.columns.get_loc("Neighborhood")

# copy Neighborhood into a temporal varable
tmp_Nh = toronto_onehot['Neighborhood']

# delete Neighborhood column
toronto_onehot.drop(toronto_onehot.columns[NhIndex], axis=1, inplace=True)

# insert the column in the index 0 of the dataframe
toronto_onehot.insert(0, 'Neighborhood', tmp_Nh)

# add neighborhood data back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

toronto_onehot.head()

,Neighborhood,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


<h1>Group categorical information by neighborhood</h1>

In [72]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Agincourt,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000
1,"Alderwood, Long Branch",0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000
2,"Bathurst Manor, Wilson Heights, Downsview North",0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.000000,0.000000,0.047619,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000
3,Bayview Village,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000
4,"Bedford Park, Lawrence Manor East",0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.043478,0.000000,...,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000
5,Berczy Park,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.017241,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000
6,"Birch Cliff, Cliffside West",0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000
7,"Brockton, Parkdale Village, Exhibition Place",0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000
8,"Business reply mail Processing Centre, South C...",0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.058824
9,"CN Tower, King and Spadina, Railway Lands, Har...",0.0,0.000000,0.071429,0.071429,0.142857,0.142857,0.142857,0.000000,0.000000,...,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000


<h1>Get top X most common venues for neighborhood</h1>

In [73]:
# function to get the most comon venues by Neighborhood# function to get the most comon venues by 

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [74]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Lounge,Latin American Restaurant,Breakfast Spot,Skating Rink,Dog Run,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center
1,"Alderwood, Long Branch",Pizza Place,Gym,Dance Studio,Coffee Shop,Skating Rink,Pharmacy,Pub,Sandwich Place,Athletics & Sports,Pool
2,"Bathurst Manor, Wilson Heights, Downsview North",Bank,Coffee Shop,Shopping Mall,Sandwich Place,Supermarket,Middle Eastern Restaurant,Sushi Restaurant,Restaurant,Deli / Bodega,Fried Chicken Joint
3,Bayview Village,Café,Bank,Chinese Restaurant,Japanese Restaurant,Deli / Bodega,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center
4,"Bedford Park, Lawrence Manor East",Restaurant,Sushi Restaurant,Coffee Shop,Italian Restaurant,Sandwich Place,Juice Bar,Café,Butcher,Pub,Liquor Store


<h1>Create clusters</h1>

In [75]:
# set number of clusters
kclusters = 4

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([3, 3, 3, 3, 3, 3, 3, 3, 3, 3], dtype=int32)

In [76]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = neighborhoods_venues_sorted.join(toronto_merged.set_index('Neighborhood'), on='Neighborhood')

# change cluster labels to integer type
#toronto_merged.astype({'Cluster Labels': 'int32'}).dtypes

toronto_merged.head() # check the last columns!

,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Postal Code,Borough,Latitude,Longitude
0,3,Agincourt,Lounge,Latin American Restaurant,Breakfast Spot,Skating Rink,Dog Run,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,M1S,Scarborough,43.794200,-79.262029
1,3,"Alderwood, Long Branch",Pizza Place,Gym,Dance Studio,Coffee Shop,Skating Rink,Pharmacy,Pub,Sandwich Place,Athletics & Sports,Pool,M8W,Etobicoke,43.602414,-79.543484
2,3,"Bathurst Manor, Wilson Heights, Downsview North",Bank,Coffee Shop,Shopping Mall,Sandwich Place,Supermarket,Middle Eastern Restaurant,Sushi Restaurant,Restaurant,Deli / Bodega,Fried Chicken Joint,M3H,North York,43.754328,-79.442259
3,3,Bayview Village,Café,Bank,Chinese Restaurant,Japanese Restaurant,Deli / Bodega,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,M2K,North York,43.786947,-79.385975
4,3,"Bedford Park, Lawrence Manor East",Restaurant,Sushi Restaurant,Coffee Shop,Italian Restaurant,Sandwich Place,Juice Bar,Café,Butcher,Pub,Liquor Store,M5M,North York,43.733283,-79.419750


<h1>Map of the clusters</h1>

In [77]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

<h1>Most common venuies per cluster</h1>

In [78]:
cluster0 = toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(2, toronto_merged.shape[1]))]] 
cluster0.sort_values(by=['2nd Most Common Venue'], inplace=True)
count_cluster0  = cluster0.groupby('1st Most Common Venue').count()
count_cluster0.sort_values(by = '2nd Most Common Venue', ascending=False ,inplace=True)
count_cluster0['Neighborhood']

1st Most Common Venue
Park                 10
Convenience Store     3
Hockey Arena          1
Name: Neighborhood, dtype: int64

In [79]:
cluster1 = toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(2, toronto_merged.shape[1]))]]  
cluster1.sort_values(by=['1st Most Common Venue'], inplace=True)
count_cluster1  = cluster1.groupby('1st Most Common Venue').count()
count_cluster1.sort_values(by = '2nd Most Common Venue', ascending=False ,inplace=True)
count_cluster1['Neighborhood']

1st Most Common Venue
Baseball Field                   1
Paper / Office Supplies Store    1
Name: Neighborhood, dtype: int64

In [80]:
cluster2 = toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(2, toronto_merged.shape[1]))]] 
cluster2.sort_values(by=['1st Most Common Venue'], inplace=True)
count_cluster2  = cluster2.groupby('1st Most Common Venue').count()
count_cluster2.sort_values(by = '2nd Most Common Venue', ascending=False ,inplace=True)
count_cluster2['Neighborhood']

1st Most Common Venue
Fast Food Restaurant    1
Name: Neighborhood, dtype: int64

In [81]:
cluster3 = toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(2, toronto_merged.shape[1]))]]  
cluster3.sort_values(by=['1st Most Common Venue'], inplace=True)
count_cluster3  = cluster3.groupby('1st Most Common Venue').count()
count_cluster3.sort_values(by = '2nd Most Common Venue', ascending=False ,inplace=True)
count_cluster3['Neighborhood']

1st Most Common Venue
Coffee Shop                  17
Grocery Store                 6
Café                          5
Clothing Store                4
Pizza Place                   4
Pharmacy                      4
Sandwich Place                3
Gym                           3
Restaurant                    2
Breakfast Spot                2
Bank                          2
Indian Restaurant             2
Bar                           2
Fast Food Restaurant          2
Portuguese Restaurant         1
Piano Bar                     1
Pool                          1
Airport Lounge                1
Pub                           1
Ramen Restaurant              1
Motel                         1
Rental Car Location           1
Shopping Mall                 1
Social Club                   1
Park                          1
Japanese Restaurant           1
Middle Eastern Restaurant     1
Metro Station                 1
Lounge                        1
Light Rail Station            1
Ice Cream Shop    